# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных**  

Набор данных находится в файле /datasets/insurance.csv

* **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи  
* **Целевой признак:** количество страховых выплат клиенту за последние 5 лет

## Загрузка данных

* Импортируем необходимые библиотеки и прочитаем файл с данными

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("/datasets/insurance.csv")

* Изучим загруженный датасет

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропуски в данных отсутствуют.

In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.duplicated().sum()

153

Удалим найденные дубликаты

In [7]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

Взглянем на целевой признак

In [8]:
df['Страховые выплаты'].value_counts()

0    4284
1     423
2     114
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Мы изучили наш датасет: пропуски отсутствуют, дубликаты удалены. Данные готовы для дальнейшей работы с ними и построения модели машинного обучения.

## Умножение матриц

*Умножим признаки на обратимую матрицу. Изменится ли качество линейной регрессии?*  

Попробуем ответить на этот вопрос и обосновать решение.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит 
из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** Для того чтобы обосновать наше решение докажем, что предсказание $a$ не изменится

Подставим вместо матрицы $X$ выражение $X*Z$, где $Z$ это обратимая матрица с некими значениями

$w_1 = ((XZ)^T XZ)^{-1} (XZ)^T y$

В формулу предсказания подставим вектор весов $w_1$ и преобразуем её:

$a_1 = XZ ((XZ)^T XZ)^{-1}(XZ)^T y = XZ (Z^TX^T XZ)^{-1}Z^T X^T y = X Z (X^TXZ)^{-1} (Z^T)^{-1} Z^T X^T y$

Продолжим преобразования, ипользуя свойства единичной матрицы $AA^{-1} = E$:

$a_1 = X Z Z^{-1}(X^TX)^{-1} (Z^T)^{-1} Z^T X^T y = X E (X^TX)^{-1} E  X^T y = X (X^TX)^{-1} X^T y =  X w$

В итоге получаем, что $a=a_1$

## Алгоритм преобразования

**Алгоритм**

В качестве преобразования умножим исходную матрицу на обратимую, состоящую из случайных чисел, воспользовавшись numpy.random.normal

**Обоснование**

В результате перемножения матриц данные клиентов приобретут вид случайного и несвязанного между собой набора цифр, что не позволит их интерпретировать, однако качество линейной регрессии не изменится (согласно п2) и модель по прежнему будет предсказывать целевой признак.

## Проверка алгоритма

Разделим наши данные на признаки и целевую переменную

In [9]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)

* Обучим модель линейной регрессии на не преобразованных данных

In [10]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print('Предсказание модели на исходных данных:', r2)

Предсказание модели на исходных данных: 0.4302010044852067


* Выполним кодирование признаков  

В соответствии с алгоритмом умножим пространство признаков на обратимую матрицу, состоящую из случайных чисел.

In [11]:
P = np.random.normal(1, size=(4, 4))
print(P)

[[-0.50330809  0.34962193 -0.86124434 -0.03559483]
 [ 0.39281657  1.52630883  0.63337186  0.22208555]
 [ 1.34247721  1.2804955   1.70592829  0.46021457]
 [-0.14616426  2.23956577  3.27106355  1.8111985 ]]


Убедимся, что наша матрица обратима

In [12]:
try:
    P_1 = np.linalg.inv(P)
    print('Матрица обратима')
except:  
    print('Матрица не обратима')

Матрица обратима


Проведем кодирование пространства признаков

In [13]:
features_p = features @ P

Посмотрим на обновленные данные клиентов

In [14]:
print(features_p.head())

              0             1             2             3
0  66602.325707  63577.744535  84642.421415  22837.523704
1  51032.057443  48731.278684  64857.681316  17500.180733
2  28203.413126  26934.668407  35842.861944   9670.946417
3  55989.256547  53433.193870  71157.052768  19199.233696
4  35049.150781  33464.018759  44541.601624  12017.783036


*Получился набор случайных цифр, отлично. Данные клиентов защищены.*

* Обучим модель линейной регрессии на преобразованных данных и сравним качество модели

In [15]:
model = LinearRegression()
model.fit(features_p, target)
predictions_p = model.predict(features_p)
r2_p = r2_score(target, predictions_p)
print('Предсказание модели на измененных данных:', r2_p)

Предсказание модели на измененных данных: 0.43020100448521215


In [16]:
# Выведем полученные результаты в виде таблицы
results = pd.DataFrame(data=[[r2, r2_p]],
                       index=['Предсказание модели'], 
                       columns=['Исходные данные', 'Измененные данные'])
results

,Исходные данные,Измененные данные
Предсказание модели,0.430201,0.430201


*Мы подтвердили на практике что качество модели не изменилось после преобразования данных, метрики R2 равны.   
При этом личные данные клиентов хорошо защищены.*